Problem statement and model description
-- 

This notebook is intended to predict the revenue of a SaaS service (subscription-based) at some period ahead.

The model assumes that the service works on two platforms (iOS and Android) and offers three basic plans (products): ```1month_trial```, ```1year_trial```, ```1year_instant``` and used to offer three outdated plans: ```1month_trial_old```, ```1year_trial_old```, ```1year_instant_old```. Plan details are described in the table below.

|product_id|subscription duration|7 day trial period|price|
|--|--|--|--|
|1month_trial|1 month|yes|\$5.99|
|1year_trial|1year|yes|\$19.99|
|1year_instant|1year|no|\$14.99|
|1month_trial_old|1 month|yes|\$2.99|
|1year_trial_old|1 year|yes|\$14.99|
|1year_instant_old|1 year|no|\$9.99|

Generally, we split the revenue into two parts: revenue from new (first-time) subscriptions and revenue from recurrent subscriptions.

New purchase and its revenue is represented in the following schema:

![title](img/activation_revenue.png)

We predict revenue from new subscriptions using the following formula:
$$Revenue_{new} =  Installs \cdot Conversion \cdot Commission \cdot \sum_{p \in products} Percent_p \cdot Price_p \cdot Conversion_p.$$
First, we predict the number of installs ```Installs``` as a time-series for each day within \[```plan_date1```, ```plan_date2```\] period with a help of fbprophet library. Then we define ```Conversion``` as conversion from installs to subscription purchase. We could simply define revenue came from a single product ```p``` as ```Price_p```, but the problem here is that some of our products have free 7 day trial period (users might cancel the subscription within the trial period which brings us no money). So we define ```Conversion_p``` as the probability that a purchase of trial product will convert to payment after the trial period ends. We need also to take into account product distribution ```Percent_p``` which denotes the share of product ```p``` among all purchases. Finally, we multiply overall revenue by ```Commission``` factor which is charged by the market (AppStore or Google Play).

As for recurrent revenue, given the historical data on previous payments, for each product (on each platform) we can estimate the probability that the ```i```-th consecutive subscription will be converted to ```(i+1)```-th. We will call it *retention function*. Having this function defined, we can estimate how many already occurred purchases will give us recurrent payments in the future.

### Model variables ###

##### Notebook variables
```plan_date1```, ```plan_date2```: global variables which stand for prediction period boundaries and are used throughout the notebook. 

##### Config variables
We set up other options and save them in ```config.json```. Here's their brief description.
<img src="img/config_sample.png" width="350">
Here's the description of some not obvious parameters.
  * install2access: ```Conversion``` from install to purchase.
  * commission: Market commission. Consists of two values: for the first year of payment and for the following years.
  * product_id: Product identifier which occurs in the historical data.
  * price: product price.
  * share: ```Percent_p``` defined above.
  * trial2purchase: ```Conversion_p``` defined above.
  * actual: Boolean flag which denotes whether the product should be taken into account while new revenue calculation (if False it will be used in recurrent revenue calculation only).

In [1]:
import holidays
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fbprophet import Prophet
plt.style.use('ggplot')
%matplotlib inline
# workaround for correct plot of time-series. See https://nextjournal.com/blog/plotting-pandas-prophet for the details
pd.plotting.register_matplotlib_converters()

In [2]:
# Defining holidays for user activity prediction
# Assume that the service works in US and UK
df_holidays_us = pd.Series(holidays.US(years=(2018, 2019, 2020, 2021))).to_frame('holiday')
df_holidays_uk = pd.Series(holidays.UK(years=(2018, 2019, 2020, 2021))).to_frame('holiday')
df_holidays = pd.concat([df_holidays_us, df_holidays_uk]).drop_duplicates()
df_holidays.index = pd.to_datetime(df_holidays.index)
df_holidays = df_holidays.sort_index().reset_index()
df_holidays = df_holidays.rename(columns={'index': 'ds'})

In [3]:
# read model configuration into global variable config
with open('config.json') as f:
    config = json.load(f)

In [4]:
# fix prediction period
plan_date1 = '2019-12-01'
plan_date2 = '2020-03-31'

In [5]:
def predict_timeseries(ts, plot=False):
    """
    The function predicts time series in the time frame [plan_date1, plan_date2]
    
    Args:
        ts: given timeseries with the data
        plot: boolean. If True shows plot with known and predicted values
    
    Returns:
        1: timeseries with combined known and predicted values
        2: fitted fbprophet model
    """
    df_prophet = ts.to_frame('y').reset_index()
    df_prophet = df_prophet.rename(columns={'date': 'ds'})
    model = Prophet(holidays=df_holidays, daily_seasonality=False, weekly_seasonality=True, yearly_seasonality=True)
    model.fit(df_prophet)
    
    days = len(pd.date_range(ts.index.max(), plan_date2))
    future = model.make_future_dataframe(periods=days)
    forecast = model.predict(future)
    if plot:
        model.plot(forecast);
        plt.title('Predicted values');
    
    df2 = forecast[(forecast['ds'] >= plan_date1) & (forecast['ds'] <= plan_date2)][['ds', 'yhat']]
    df2 = df2.set_index('ds')['yhat'].astype(int)
    df2.index.name = 'date'
    return df2


def predict_new_revenue(installs):
    """
    The function takes installs as input and predicts revenue size gained by these new users.
    
    Args:
        installs: pandas.TimeSeries with installs values. Name attribute of installs must be [iOS|Android|Web]
    
    Returns:
        1: pandas.DataFrame with pandas.MultiIndex columns. Products are at the second level, at the first level there are:
            accesses (total amount of purchases, including trials),
            purchases (total amount of subscriptions ordered, excluding trials),
            revenue (revenue given by ordered subscriptions).
        2: pandas.DataFrame with columns date, product_id, duration, billing_cycle, purchases.
        This DataFrame will be passed as input for further recurrent revenue prediction.
    """
    platform = installs.name
    conv = get_isntall2access(config[platform]['install2access'])
    products = [p for p in config[platform]['products'] if 'actual' in p and p['actual'] is True]
    commission = config[platform]['commission'][0]
    
    res = []
    for day, inst in installs.iteritems():
        for p in products:
            accesses = inst * conv[day] * p['share']
            purchases = accesses * p['trial2purchase']
            price = p['price'][0] if isinstance(p['price'], list) else p['price']
            revenue = purchases * price * commission
            res.append([day, p['product_id'], accesses, purchases, revenue, revenue / inst])
    res = pd.DataFrame(res, columns=['date', 'product_id', 'accesses', 'purchases', 'revenue', 'arpu'])
    res['accesses'] = res['accesses'].astype(int)
    res['purchases'] = res['purchases'].astype(int)
    res['revenue'] = res['revenue'].astype(int)
    
    res = res.pivot(index='date', columns='product_id')
    res[('installs', 'total')] = installs.astype(int)
    res[('accesses', 'total')] = res['accesses'].sum(axis=1)
    res[('purchases', 'total')] = res['purchases'].sum(axis=1)
    res[('revenue', 'total')] = res['revenue'].sum(axis=1)
    
    # временно переименовывем колонки, чтобы потом отсортировать в нужном порядке     
    new_names = ['installs', 'accesses', 'purchases', 'revenue', 'arpu']
    new_names1 = dict((x, str(i) + '_' + x) for i, x in enumerate(new_names))
    new_names2 = dict((str(i) + '_' + x, x) for i, x in enumerate(new_names))
    res = res.rename(columns=new_names1, level=0)
    res = res.sort_index(axis=1)
    res = res.rename(columns=new_names2, level=0)
    
    res2 = res['purchases'].unstack().swaplevel().to_frame('purchases').reset_index()
    res2['duration'] = res2['product_id'].map(lambda x: 1 if 'month' in x else 12)
    res2['billing_cycle'] = 1

    return res, res2


def get_isntall2access(conv_config):
    """
    Auxiliary function for conversion dictionary config[platform]['install2access'] to daily conversion dictionary,
    i.e. keys are dates, values are conversion.

    Args:
        conv_config: config subdictionary contained install2access conversion.
    
    Returns:
        If config[platform]['install2access'] is dictionary, it returns converted dictionary as it's described above.
        Keys are of pandas.Timestamp type.
        Otherwise returns config[platform]['install2access'] value itself.
    """
    
    custom_periods = []
    if isinstance(conv_config, dict):    
        for key in conv_config.keys():
            if key != "default":
                date1 = pd.to_datetime(key)
                date2 = date1 + pd.DateOffset(months=1) - pd.DateOffset(days=1)
                for date in pd.date_range(str(date1), str(date2)):
                    custom_periods.append([date, conv_config[key]])
        default_conv = conv_config['default']
    else:
        default_conv = conv_config
                    
    custom_periods = pd.DataFrame(custom_periods, columns=['date', 'install2access'])
        
    conv = []
    for date in pd.date_range(plan_date1, plan_date2):
        conv.append([date, default_conv])
    conv = pd.DataFrame(conv, columns=['date', 'install2access'])

    conv = pd.merge(conv, custom_periods, how='outer', on='date', indicator=True)
    merge_conversions = lambda row: row['install2access_x'] if row['_merge'] == 'left_only' else row['install2access_y']
    conv['install2access'] = conv.apply(merge_conversions, axis=1)
    conv = conv[['date', 'install2access']]
    conv = dict(zip(conv['date'], conv['install2access']))
    return conv


def get_retention(group):
    """
    The function returns subscription retention rate.
    
    Args:
        group: pandas.DataFrame containing data on one particular product.
            Must have columns date, duration, billing_cycle, purchases
        
    Returns:
        pandas.Series with retention values. Index is billing_cycle (starting with 1).
    """
    duration = group.name[2]
    min_date, max_date = group['date'].min(), plan_date2 #group['date'].max()
    # maximum possible billing_cycle for a given product within the dataset
    max_bc = int((len(pd.date_range(min_date, max_date, freq='30D'))) / duration) + 1

    retention = [1.]
    for bc in range(1, max_bc):
        # leave those subscriptions only which had a chance to convert to the next billing cycle.
        # I.e. if plan_date1 = 2019-12-01 then for monthly subscriptions we leave the data before 2019-11-01,
        # and for annual subsciptions -- before 2018-12-01
        mask = (group['billing_cycle'] == bc) &\
               (group['date'] < pd.to_datetime(plan_date1) - pd.DateOffset(months=duration))
        # if only few subscribers left at the current billing_cycle step (or no subscribers at all)
        # we consider such billing cycle as empty and define retention values at these points artificially
        
        purch_from = group[mask]['purchases'].sum()
        purch_to = group[group['billing_cycle'] == bc + 1]['purchases'].sum()
        if purch_from < 30 or purch_to < 30:
            if duration == 1 and bc > 12:
                r = 0.9
            elif duration == 1 and bc <= 12:
                r = 0.8
            elif duration == 12:
                r = 0.5
            else:
                raise ValueError('Unknown product duration value (expectected 1 or 12)')
        else:
            r = purch_to / purch_from
        retention.append(r)
        
    retention = np.cumprod(retention)
    index = pd.Index(range(1, len(retention) + 1), name='billing_cycle')
    return pd.Series(retention, index)


def predict_recurrent_revenue(df, renention):
    """
    The function calculates expected revenue from recurrent subscription within [plan_date`, plan_date2] period
    
    Args:
        df: pandas.DataFrame containing historical data on previous subscriptions. Must have the following columns:
            date, product_id, duration, billing_cycle, purchases.
        retention: pandas.Series obtained at the previous step as an output of get_retention function.
    """
    res = []
    # condsider monthly recurrent subscriptions first.
    # within prediction period [plan_date1, plan_date2] only those subscriptions might be prolonged
    # which occured previously within [plan_date1 - 1 month; plan_date1) period.
    # Thus we leave those subscriptions only and calculate what amount of them will be prolonged within [plan_date1, plan_date2]
    mask_month = (df['date'] >= pd.to_datetime(plan_date1) - pd.DateOffset(months=1)) &\
                 (df['date'] < pd.to_datetime(plan_date1)) &\
                 (df['duration'] == 1)
    # Same idea for annual subscriptions. Leave only those from [plan_date1 - 1 year; plan_date2 - 1 year) 
    # which have a chance to be prolonged within [plan_Date1, plan_date2].
    mask_year = (df['date'] >= pd.to_datetime(plan_date1) - pd.DateOffset(years=1)) &\
                 (df['date'] <= pd.to_datetime(plan_date2) - pd.DateOffset(years=1)) &\
                 (df['duration'] == 12)
    df_month, df_year = df[mask_month], df[mask_year]
    
    for i, row in df_month.iterrows():
        date, bc, product_id, duration = row['date'], row['billing_cycle'], row['product_id'], row['duration']
        month_offset = 1
        next_date = date + pd.DateOffset(months=month_offset)
        while next_date <= pd.to_datetime(plan_date2):
            retention_to = retention[(row['platform'], product_id, duration, bc + month_offset)]
            retention_from = retention[(row['platform'], product_id, duration, bc)]
            pred_purchases = row['purchases'] * retention_to / retention_from
            res.append([next_date, row['platform'], product_id, duration, bc + month_offset, pred_purchases])
            
            month_offset += 1
            next_date = date + pd.DateOffset(months=month_offset)
    
    for i, row in df_year.iterrows():
        date, bc, product_id, duration = row['date'], row['billing_cycle'], row['product_id'], row['duration']
        next_date = date + pd.DateOffset(years=1)
        retention_to = retention[(row['platform'], product_id, duration, bc + 1)]
        retention_from = retention[(row['platform'], product_id, duration, bc)]
        pred_purchases = row['purchases'] * retention_to / retention_from
        res.append([next_date, row['platform'], product_id, duration, bc + 1, pred_purchases])
    
    res = pd.DataFrame(res, columns=['date', 'platform', 'product_id', 'duration', 'billing_cycle', 'purchases'])
    
    # Считаем выручку на основе предсказанных доступов
    commission = config[platform]['commission']
    prices = dict((p['product_id'], p['price']) for p in config[platform]['products'])
    res['price'] = res['product_id'].map(prices)
    
    def get_commission(row):
        if row['duration'] == 12 and row['billing_cycle'] > 1:
            return commission[1]
        if row['duration'] == 1 and row['billing_cycle'] > 12:
            return commission[1]
        return commission[0]    
    res['commission'] = res.apply(get_commission, axis=1)
    
    res['purchases'] = res['purchases'].map(lambda x: x if x >= 1 else 0)
    res['revenue'] = res['purchases'] * res['price'] * res['commission']

    return res

In [6]:
platforms = ['iOS', 'Android']
revenue_types = ['new', 'recurrent', 'total']
column_index = pd.MultiIndex.from_product([platforms, revenue_types], names=['platform', 'revenue_type'])
plan = pd.DataFrame(columns=column_index)

New revenue
--

### Installs prediction

In [7]:
df = pd.read_csv('data/installs.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [8]:
installs = pd.DataFrame()
installs['iOS'] = predict_timeseries(df['iOS'])
installs['Android'] = predict_timeseries(df['Android'])
installs['Web'] = predict_timeseries(df['Web'])

In [9]:
installs.head()

,iOS,Android,Web
date,,,
2019-12-01,340827,601777,487974
2019-12-02,311833,575541,513486
2019-12-03,306263,569199,510160
2019-12-04,305887,569306,507308
2019-12-05,311931,579234,506751


### New revenue prediction

In [10]:
new_recur = {}
for platform in ['iOS', 'Android']:
    new_purchases, new_recur[platform] = predict_new_revenue(installs[platform])
    plan[(platform, 'new')] = new_purchases[('revenue', 'total')]
    new_recur[platform]['platform'] = platform
# DataFrame for storing predicted new purchases. It will be used later at recurring subscriptions processing.
new_recur = pd.concat([new_recur['iOS'], new_recur['Android']])

In [11]:
plan.head()

platform         iOS                 Android                
revenue_type     new recurrent total     new recurrent total
date                                                        
2019-12-01    442889       NaN   NaN  447005       NaN   NaN
2019-12-02    405212       NaN   NaN  427517       NaN   NaN
2019-12-03    397975       NaN   NaN  422806       NaN   NaN
2019-12-04    397486       NaN   NaN  422886       NaN   NaN
2019-12-05    405340       NaN   NaN  430260       NaN   NaN

Recurrent revenue
--

### Subscription retention

In [12]:
# Loading historical data
df = pd.read_csv('data/recurrent.csv')
df['date'] = pd.to_datetime(df['date'])

In [13]:
df.head()

,date,platform,product_id,duration,billing_cycle,purchases
0,2018-02-14,iOS,1month_trial_old,1,1,10
1,2018-02-15,iOS,1month_trial_old,1,1,5
2,2018-02-16,iOS,1month_trial_old,1,1,5
3,2018-02-17,iOS,1month_trial_old,1,1,13
4,2018-02-18,iOS,1month_trial_old,1,1,11


In [14]:
retention = df.groupby(['platform', 'product_id', 'duration']).apply(get_retention)

We need to add new purchases predicted at the previous step to historical data, because they also might convert to recurrent subscriptions within prediction period (especially monthly).

In [15]:
df = pd.concat([df, new_recur], sort=True)

In [16]:
df.head()

,billing_cycle,date,duration,platform,product_id,purchases
0,1,2018-02-14,1,iOS,1month_trial_old,10
1,1,2018-02-15,1,iOS,1month_trial_old,5
2,1,2018-02-16,1,iOS,1month_trial_old,5
3,1,2018-02-17,1,iOS,1month_trial_old,13
4,1,2018-02-18,1,iOS,1month_trial_old,11


In [17]:
recur = predict_recurrent_revenue(df, retention)

In [18]:
recur.head()

,date,platform,product_id,duration,billing_cycle,purchases,price,commission,revenue
0,2019-12-01,iOS,1month_trial_old,1,2,1.364789,2.99,0.7,2.856504
1,2020-01-01,iOS,1month_trial_old,1,3,1.089543,2.99,0.7,2.280414
2,2020-02-01,iOS,1month_trial_old,1,4,0.000000,2.99,0.7,0.000000
3,2020-03-01,iOS,1month_trial_old,1,5,0.000000,2.99,0.7,0.000000
4,2019-12-01,iOS,1month_trial_old,1,4,0.000000,2.99,0.7,0.000000


Last step. Summing up the revenue over all revenue types and platforms.

In [19]:
for platform in ('iOS', 'Android'):
    # Sum up all recurrent payments occured at the same day
    plan[(platform, 'recurrent')] = recur.groupby('date')['revenue'].sum().astype(int)
    # sum up new and recurrent revenue as total
    plan[(platform, 'total')] = plan[(platform, 'new')] + plan[(platform, 'recurrent')]
    
for revenue_type in (['new', 'recurrent', 'total']):
    plan[('total', revenue_type)] = plan[('iOS', revenue_type)] + plan[('iOS', revenue_type)]

Finally, here's the result. We've predicted new, recurrent and total revenue for each platform and each day within prediction period.

In [20]:
with pd.option_context('display.max_rows', 10):
    display(plan)

platform         iOS                   Android                     total  \
revenue_type     new recurrent   total     new recurrent   total     new   
date                                                                       
2019-12-01    442889     12522  455411  447005     12522  459527  885778   
2019-12-02    405212     12093  417305  427517     12093  439610  810424   
2019-12-03    397975      9697  407672  422806      9697  432503  795950   
2019-12-04    397486     10085  407571  422886     10085  432971  794972   
2019-12-05    405340     11247  416587  430260     11247  441507  810680   
...              ...       ...     ...     ...       ...     ...     ...   
2020-03-27    283553     12958  296511  359445     12958  372403  567106   
2020-03-28    313779     14678  328457  384816     14678  399494  627558   
2020-03-29    292476     13906  306382  352580     13906  366486  584952   
2020-03-30    262605     16303  278908  337693     16303  353996  525210   
2020-03-31    256270     16547  272817  333841     16547  350388  512540   

platform                        
revenue_type recurrent   total  
date                            
2019-12-01       25044  910822  
2019-12-02       24186  834610  
2019-12-03       19394  815344  
2019-12-04       20170  815142  
2019-12-05       22494  833174  
...                ...     ...  
2020-03-27       25916  593022  
2020-03-28       29356  656914  
2020-03-29       27812  612764  
2020-03-30       32606  557816  
2020-03-31       33094  545634  

[122 rows x 9 columns]